In [1]:
from graphsage.model import load_cora

In [2]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable

import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict

from graphsage.encoders import Encoder
from graphsage.aggregators import MeanAggregator

In [ ]:
class UnsupervisedGraphSage(nn.Module):

    def __init__(self, enc, degree_list):
        super(UnsupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.BCELoss()
        wt = np.power(degree_list, 0.75)
        wt = wt / wt.sum()
        self.weights = torch.FloatTensor(wt)
        
    def negative_sample(self, number_of_neg_sample):
        return torch.multinomial(self.weights, number_of_neg_sample, 
                                 replacement=True)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        return embeds
    
    def affinity(self, input_1, input_2):
        output_1 = torch.nn.functional.normalize(self.forward(input_1))
        output_2 = torch.nn.functional.normalize(self.forward(input_2))
        aff = torch.sum((output_1 * output_2), dim=1)
        
        return output_1, aff
    
    def neg_affinity(self, output_1, neg_samples):
        neg_output = torch.nn.functional.normalize(self.forward(neg_samples))
        neg_aff = torch.mm(output_1.t(),neg_output)
        
        return neg_aff

    def loss(self, edges, neg_samples):
        input_1 = [edge[0] for edge in edges]
        input_2 = [edge[1] for edge in edges]
        
        output_1, aff = self.affinity(input_1, input_2)
        neg_aff = self.neg_affinity(output_1, neg_samples)
        
        total_loss = 0
        total_loss += - torch.sum(torch.log(torch.sigmoid(aff)))
        total_loss += - len(neg_samples) * torch.sum((torch.log(torch.sigmoid(-neg_aff))))
        
        return total_loss
    
    def compare_loss(self, edges):
        input_1 = [edge[0] for edge in edges]
        input_2 = [edge[1] for edge in edges]
        
        _, aff = self.affinity(input_1, input_2)
        total_loss = - torch.mean(torch.log(torch.sigmoid(aff)))
        
        return total_loss

In [ ]:
num_nodes = 2708
feat_data, labels, adj_lists = load_cora()
features = nn.Embedding(2708, 1433)
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
agg1 = MeanAggregator(features, cuda=True)
enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)
enc1.num_samples = 5
enc2.num_samples = 5
rand_indices = np.random.permutation(num_nodes)
test = rand_indices[:1000]
val = rand_indices[1000:1500]
train = list(rand_indices[1500:])
train_degree_list = [len(adj_lists[node]) for node in train]
train_edges = [(row, node) for row in train for node in adj_lists[row] if node in train]
val_edges = [(row, node) for row in val for node in adj_lists[row] if node not in test]
graphsage = UnsupervisedGraphSage(enc2, train_degree_list)
optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.0001)
batch_size = 30
number_of_neg_sample = 20
times = []
for batch in range(1000):
    batch_edges = train_edges[:batch_size]
    neg_samples = graphsage.negative_sample(number_of_neg_sample)
    random.shuffle(train_edges)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss(batch_edges, neg_samples)
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time-start_time)
    if batch % 20 == 0:
        loss_train =  graphsage.compare_loss(train_edges)
        loss_val = graphsage.compare_loss(val_edges)
        print(batch, loss.data, loss_train.data, loss_val.data)

In [ ]:
val = rand_indices[1000:1500]

In [ ]:
val_edges = [(row, node) for row in val for node in adj_lists[row] if node not in test]